In [1]:
import requests
import pandas as pd
import tqdm
import bs4
import json
import matplotlib.pyplot as plt
from adjustText import adjust_text
from pathlib import Path
import time

In [2]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

In [3]:
from nba_api.stats.endpoints.playbyplay import PlayByPlay
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs

In [4]:
path = Path('pbp_data/2023-24.csv')
data_frame = None
if path.exists():
    data_frame = pd.read_csv(str(path))
    data_frame.loc[:, 'GAME_ID'] = [f"00{item}" for item in data_frame.GAME_ID.astype(str)]

In [5]:
season = '2023-24'

In [6]:
game_finder = LeagueGameFinder(season_nullable=season, league_id_nullable='00', season_type_nullable='Regular Season')
df = game_finder.get_data_frames()[0]

In [7]:
season = '2023-24'
print(f"Processing {season}")
pbp_dfs = []
game_finder = LeagueGameFinder(season_nullable=season, league_id_nullable='00', season_type_nullable='Regular Season')
df = game_finder.get_data_frames()[0]
game_ids = df.GAME_ID.unique()
for game_id in tqdm.tqdm(game_ids):
    if data_frame is not None and game_id in data_frame.GAME_ID.tolist():
        continue
    while True:
        try:
            pbp_df = PlayByPlayV2(game_id=game_id).get_data_frames()[0]
            break
        except Exception as e:
            print(f"Error occured: {e}, Game ID: {game_id}, retrying")
            time.sleep(0.1)
    pbp_dfs.append(pbp_df)
    time.sleep(0.25)

concated_downloaded = pd.concat(pbp_dfs)

Processing 2023-24


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 279/279 [00:25<00:00, 11.13it/s]


In [8]:
full_df = pd.concat([data_frame, concated_downloaded])

In [9]:
full_df.loc[:, "PCTIMESECONDS"] = [720 - (int(item.split(":")[0]) * 60 + int(item.split(":")[1])) for item in full_df.PCTIMESTRING.tolist()]

In [10]:
full_df = full_df.fillna({
    "HOMEDESCRIPTION": "",
    "NEUTRALDESCRIPTION": "",
    "VISITORDESCRIPTION": ""
})

In [11]:
full_df.to_csv('pbp_data/2023-24.csv', index=False)

## Work

In [21]:
path = Path('pbp_data/2022-23.csv')
last_df = None
if path.exists():
    last_df = pd.read_csv(str(path))
    last_df.loc[:, 'GAME_ID'] = [f"00{item}" for item in last_df.GAME_ID.astype(str)]

In [22]:
len(last_df.GAME_ID.unique())

976